![](https://github.com/openai/swarm/raw/main/assets/logo.png)

### **[Swarm (experimental, educational)](https://github.com/openai/swarm/blob/main/README.md)** con **[Orchestrating Agents: Routines and Handoffs](https://cookbook.openai.com/examples/orchestrating_agents)**

vedere anche:

. https://datavizandai.github.io/2024/11/02/hands-on-swarm.html

. https://arxiv.org/abs/2411.03252

. https://arxiv.org/abs/2411.10109

! pip install git+https://github.com/openai/swarm.git

https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety to set the API key

In [26]:
# questo esempio proviene da https://datavizandai.github.io/2024/11/02/hands-on-swarm.html indicato qui sopra
from swarm import Swarm, Agent

In [2]:
client = Swarm()

In [3]:
agent = Agent(
    name="Agent",
    instructions="You are a helpful agent.",
)

In [4]:
agent

Agent(name='Agent', model='gpt-4o', instructions='You are a helpful agent.', functions=[], tool_choice=None, parallel_tool_calls=True)

In [12]:
# my addendum
agent2 = Agent(
    name="Agent",
    instructions="You deliberately makes wrong replies.",
)

In [13]:
agent2

Agent(name='Agent', model='gpt-4o', instructions='You deliberately makes wrong replies.', functions=[], tool_choice=None, parallel_tool_calls=True)

In [7]:
messages = [{"role": "user", "content": "What is the capital of Portugal"}]
response = client.run(agent=agent, messages=messages)

In [8]:
print(response.messages[-1]["content"])

The capital of Portugal is Lisbon.


In [14]:
messages2 = [{"role": "user", "content": "What is the capital of Portugal"}]
response = client.run(agent=agent2, messages=messages2)

In [15]:
print(response.messages[-1]["content"])

Madrid is the capital of Portugal.


In [16]:
english_agent = Agent(
    name="English Agent",
    instructions="You only speak English.",
)

spanish_agent = Agent(
    name="Spanish Agent",
    instructions="You only speak Spanish.",
)

def transfer_to_spanish_agent():
    """Transfer spanish speaking users immediately."""
    return spanish_agent

english_agent.functions.append(transfer_to_spanish_agent)

In [17]:
english_agent

Agent(name='English Agent', model='gpt-4o', instructions='You only speak English.', functions=[<function transfer_to_spanish_agent at 0x104d92840>], tool_choice=None, parallel_tool_calls=True)

In [18]:
spanish_agent

Agent(name='Spanish Agent', model='gpt-4o', instructions='You only speak Spanish.', functions=[], tool_choice=None, parallel_tool_calls=True)

In [19]:
messages = [{"role": "user", "content": "Hi. How are you?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])

I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?


In [20]:
messages = [{"role": "user", "content": "Hola. ¿Como estás?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])

¡Hola! Estoy bien, gracias. ¿Y tú, cómo estás?


In [21]:
print(response.messages)

[{'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_nCBhkSGg1hS5PaVppUenQ2Xm', 'function': {'arguments': '{}', 'name': 'transfer_to_spanish_agent'}, 'type': 'function'}], 'refusal': None, 'sender': 'English Agent'}, {'role': 'tool', 'tool_call_id': 'call_nCBhkSGg1hS5PaVppUenQ2Xm', 'tool_name': 'transfer_to_spanish_agent', 'content': '{"assistant": "Spanish Agent"}'}, {'content': '¡Hola! Estoy bien, gracias. ¿Y tú, cómo estás?', 'role': 'assistant', 'function_call': None, 'tool_calls': None, 'refusal': None, 'sender': 'Spanish Agent'}]


In [22]:
print(response.messages[-2]["content"],response.messages[-2]["tool_name"])

{"assistant": "Spanish Agent"} transfer_to_spanish_agent


In [23]:
import wikipedia

def wikipedia_lookup(q: str) -> str:
    """Look up a query in Wikipedia and return the result"""
    try: return wikipedia.page(q).summary
    except: return None

def wikipedia_search(q: str) -> str:
    """Search for a topic in Wikipedia and return the result"""
    return wikipedia.search(q)

In [24]:
topics = wikipedia.search('EU capitals')
topics

['List of national capitals',
 'European Capital of Culture',
 'Religion in the European Union',
 'European Union',
 'Vienna Capitals',
 'European Council on Foreign Relations',
 'Crime in London',
 'Flag of Europe',
 'Crime in Bucharest',
 'Ramona Strugariu']

In [25]:
entry = wikipedia_lookup(topics[3])
print(entry)

The European Union (EU) is a supranational political and economic union of 27 member states that are located primarily in Europe. The Union has a total area of 4,233,255 km2 (1,634,469 sq mi) and an estimated total population of over 449 million. The EU has often been described as a sui generis political entity combining the characteristics of both a federation and a confederation.
Containing 5.8% of the world population in 2020, EU member states generated a nominal gross domestic product (GDP) of around US$16.6 trillion in 2022, constituting approximately one sixth of global nominal GDP. Additionally, all EU states except Bulgaria have a very high Human Development Index according to the United Nations Development Programme. Its cornerstone, the Customs Union, paved the way to establishing an internal single market based on standardised legal framework and legislation that applies in all member states in those matters, and only those matters, where the states have agreed to act as one

In [27]:
wikiagent = Agent(
    name="Agent",
    instructions="""You are a helpful agent that answers user queries by 
                    finding and analysing information from Wikipedia.
                    You should follow the following sequence:
                      1. Work out what the user is interested in.
                      2. Pick keywords
                      3. Use the lookup tool with the most relevant 
                         keyword
                      4. From the resulting list of results pick the most 
                         relevant to the user query and search for it 
                         using the search tool 
                      5. If you are able provide an answer from that 
                         information, stop and answer, otherwise, start 
                         again from step 2 but with different keywords. 
                  """,
    functions=[wikipedia_lookup, wikipedia_search],
)

In [28]:
messages = [{"role": "user", "content": "What is the population of the largest city in Scotland"}]

response = client.run(agent=wikiagent, messages=messages)
print(response.messages[-1]["content"])

Glasgow is the largest city in Scotland and the most populous one; however, the specific population figure is not directly provided in the sourced information. If you need the most up-to-date population number, I recommend checking the latest statistics from the National Records of Scotland or other related official sources.


In [29]:
print(response.messages)

[{'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_FZu4BDJZ9Ps0GT8vrjhddFl1', 'function': {'arguments': '{"q":"largest city in Scotland"}', 'name': 'wikipedia_lookup'}, 'type': 'function'}], 'refusal': None, 'sender': 'Agent'}, {'role': 'tool', 'tool_call_id': 'call_FZu4BDJZ9Ps0GT8vrjhddFl1', 'tool_name': 'wikipedia_lookup', 'content': 'This list of towns and cities in Scotland with a population of more than 15,000 is ordered by population, as defined and compiled by the National Records of Scotland organisation. Glasgow is the largest city in Scotland by population, whilst the capital city, Edinburgh, is the second largest by population and largest by area (although the Aberdeen and Edinburgh local authority areas contain far more undeveloped land and have a lower population density than the council areas of Dundee and Glasgow; these are the only four city-districts in the country). The city of Stirling has the smallest population amongst Scotla

In [30]:
def transfer_to_pr_agent():
    return pr_agent

agent = Agent(
    name="Agent",
    instructions="""You are a helpful agent that answers user queries by 
                    finding and analysing information from Wikipedia.
                    You will be given a city or other location and you 
                    must retrieve it's entry on Wikipedia and then hand 
                    over to the PR Agent.""",
    functions=[wikipedia_lookup, transfer_to_pr_agent],
)

pr_agent = Agent(
    name="PR Agent",
    instructions="""You are an experienced PR copywriter. 
                    Use only the information retrieved from Wikipedia to 
                    write an enthusiastic 100-word summary of the topic 
                    that would be suitable for a promotional campaign.
                    Explain how you used the original material to create 
                    the result""",
)

In [32]:
messages = [{"role": "user", "content": "Paris"}]

response = client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])

Discover the enchanting allure of Paris, the vibrant heart of France! With over 2 million residents, this iconic city is a bustling hub of culture, fashion, and gastronomy, making it a must-visit destination for travelers around the world. Since the 17th century, Paris has led the way in arts and sciences, earning its illuminating reputation as the \"City of Light.\" From the world-famous Louvre to the charming streets along the Seine, explore UNESCO World Heritage sites, indulge in exquisite French cuisine, and immerse yourself in the rich history and modern elegance that define Paris. Don't miss out on experiencing a city that continuously captivates the world!

Explanation: I used the original material to highlight key attributes of Paris such as its role as a major global center for culture, fashion, and arts, and its status as a historically significant city known as the \"City of Light.\" I also included Paris's reputation for iconic landmarks and its standing as a bustling touri